<a href="https://colab.research.google.com/github/Ava-00/Causal-Inference-and-Algorithmic-Fairness/blob/main/Kusner_Paper_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pystan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(0)

# Number of samples
n_samples = 1000

# Create race as categorical values without predefined proportions
races = ["White", "Hispanic", "Black", "Asian", "Amerind", "Other"]
data = pd.DataFrame({
    "race": np.random.choice(races, size=n_samples),
    "sex": np.random.choice([1, 0], size=n_samples, p=[0.5, 0.5]),
    "region_first": np.random.choice(["GL", "MS", "NE", "SE", "SW", "W"], size=n_samples),
    "UGPA": np.random.normal(3.5, 0.4, n_samples).round(1),
    "ZFYA": np.random.normal(0, 1, n_samples).round(2),  # Normal distribution centered around 0 for simplicity
    "sander_index": np.random.uniform(0.6, 0.8, n_samples).round(10),
    "first_pf": 1.0
})

data = pd.get_dummies(data, columns=["race", "region_first"], drop_first=False)

data["LSAT"] = np.random.poisson(35, n_samples).round(1)

sensitive_columns = [col for col in data.columns if col.startswith("race_") or col.startswith("region_first_") or col == "sex"]
a = data[sensitive_columns].values

# Create input data for the model
stan_data = {
    'N': n_samples,
    'K': a.shape[1],
    'a': a,
    'ugpa': data['UGPA'].values,
    'lsat': data['LSAT'].values.astype(int),
    'zfya': data['ZFYA'].values
}


train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)

# Model Configuration
X_train_unfair = train_data[sensitive_columns + ["LSAT", "UGPA"]]
X_test_unfair = test_data[sensitive_columns + ["LSAT", "UGPA"]]
y_train = train_data["ZFYA"]
y_test = test_data["ZFYA"]

model_unfair = LinearRegression().fit(X_train_unfair, y_train)
pred_unfair_test = model_unfair.predict(X_test_unfair)
rmse_unfair_test = np.sqrt(mean_squared_error(y_test, pred_unfair_test))
print(f"Unfair Model RMSE: {rmse_unfair_test}")

# Model Unaware (sensitive features not included)
X_train_unaware = train_data[["LSAT", "UGPA"]]
X_test_unaware = test_data[["LSAT", "UGPA"]]

model_unaware = LinearRegression().fit(X_train_unaware, y_train)
pred_unaware_test = model_unaware.predict(X_test_unaware)
rmse_unaware_test = np.sqrt(mean_squared_error(y_test, pred_unaware_test))
print(f"Unaware Model RMSE: {rmse_unaware_test}")

# Fair model (accounting for sensitive features and performing multiple abduction steps)
stan_model_code = """
data {
  int<lower=0> N;
  int<lower=0> K;
  matrix[N, K] a;
  vector[N] ugpa;
  vector[N] lsat;
  vector[N] zfya;
}
parameters {
  real ugpa0;
  real eta_u_ugpa;
  vector[K] eta_a_ugpa;
  real lsat0;
  real eta_u_lsat;
  vector[K] eta_a_lsat;
  real sigma_g;
  vector[N] u;
}
model {
  zfya ~ normal(ugpa + lsat + u, sigma_g);
}
"""

Unfair Model RMSE: 0.9925913708802588
Unaware Model RMSE: 0.9862696224094137


In [ ]:
# Create training data in the format required for PyStan
import stan
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error

# Prepare the data for PyStan
stan_data_train = {
    'N': len(train_data),
    'K': len(sensitive_columns),
    'a': train_data[sensitive_columns].values,
    'ugpa': train_data['UGPA'].values,  # Target variable UGPA
    'lsat': train_data['LSAT'].values,  # Target variable LSAT
    'zfya': train_data['ZFYA'].values   # Additional target variable ZFYA
}

# # Interesting way to sample! Fit a Bayesian Model using pystan and sample from the posterior distribution
# posterior = stan.build(stan_model_code, data=stan_data_train)
# fit = posterior.sample(num_chains=1, num_samples=2000)  # Sample from posterior

# # Extract the model parameters
# # fit_results = fit.to_frame()  # Extract results as a pandas DataFrame
# # u_cols = [col for col in fit_results.columns if col.startswith('u[')]
# # u_train = fit_results[u_cols].mean(axis=0)

# Train deterministic model for fair approach
residual_ugpa_model = LinearRegression().fit(train_data[sensitive_columns], train_data["UGPA"])
train_data["resid_UGPA"] = train_data["UGPA"] - residual_ugpa_model.predict(train_data[sensitive_columns])

residual_lsat_model = LinearRegression().fit(train_data[sensitive_columns], train_data["LSAT"])
train_data["resid_LSAT"] = train_data["LSAT"] - residual_lsat_model.predict(train_data[sensitive_columns])

# Fair deterministic model on residuals
X_fair_train = train_data[["resid_UGPA", "resid_LSAT"]]
model_fair_det = LinearRegression().fit(X_fair_train, y_train)

# Calculate RMSE for test data using fair deterministic model
test_data["resid_UGPA"] = test_data["UGPA"] - residual_ugpa_model.predict(test_data[sensitive_columns])
test_data["resid_LSAT"] = test_data["LSAT"] - residual_lsat_model.predict(test_data[sensitive_columns])
X_fair_test = test_data[["resid_UGPA", "resid_LSAT"]]

# Predict and calculate RMSE
pred_fair_test = model_fair_det.predict(X_fair_test)
rmse_fair_test = np.sqrt(mean_squared_error(y_test, pred_fair_test))
print(f"Fair Deterministic Model RMSE: {rmse_fair_test}")

Fair Deterministic Model RMSE: 0.9864374191531695
